In [ ]:
from copy import deepcopy

import nd2reader
import numpy as np
import zarr
from numcodecs import Blosc, Delta
from tqdm import tnrange, tqdm_notebook

# 171018

In [ ]:
pass

# 171214

In [ ]:
frames = nd2reader.ND2Reader(
    "/home/jqs1/scratch/fidelity/171214/transcriptionerror_timelapse.nd2"
)

In [ ]:
compressor = Blosc(cname="zstd", clevel=5, shuffle=Blosc.SHUFFLE, blocksize=0)
store = zarr.DirectoryStore(
    "/home/jqs1/scratch/fidelity/171214/transcriptionerror_timelapse.zarr"
)
root_group = zarr.open_group(store=store)
raw_group = root_group.require_group("raw")
meta = deepcopy(frames.metadata)
meta["date"] = meta["date"].isoformat()
raw_group.attrs["metadata"] = meta
for v in tnrange(frames.sizes["v"]):
    ary = raw_group.require_dataset(
        "{:d}".format(v),
        shape=[frames.sizes[n] for n in "ctyx"],
        chunks=(1, 1, 512, 512),
        dtype="u2",
        order="C",
        compressor=compressor,
    )
    if "ingested" in ary.attrs and ary.attrs["ingested"]:
        continue
    ary.attrs["ingested"] = False
    ary.attrs["metadata"] = meta
    for c in range(frames.sizes["c"]):
        for t in tnrange(frames.sizes["t"]):
            ary[c, t, :, :] = frames.get_frame_2D(c=c, t=t, v=v)
    ary.attrs["ingested"] = True